<a href="https://colab.research.google.com/github/ProfeLuisTic1986/Quiz_listas/blob/main/quizz_Listas%26Flet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import flet as ft

# Paleta de colores moderna
PRIMARY = ft.Colors.DEEP_PURPLE_400
PRIMARY_DARK = ft.Colors.DEEP_PURPLE_700
ACCENT = ft.Colors.AMBER_400
OK = ft.Colors.GREEN_400
BAD = ft.Colors.RED_400
TEXT_PRIMARY = ft.Colors.with_opacity(0.95, ft.Colors.WHITE)
TEXT_DARK = ft.Colors.GREY_900

def main(page: ft.Page):
    page.title = "Quiz: Historia de la Informática"
    page.theme_mode = "light"
    page.padding = 0
    page.window_min_width = 480
    page.window_min_height = 720

    SHUFFLE_QUESTIONS = True
    SHUFFLE_OPTIONS = True

    preguntas = [
        {"pregunta": "¿Quién es considerado el 'padre de la computación'?",
         "opciones": ["Charles Babbage", "Alan Turing", "Bill Gates", "Konrad Zuse"],
         "respuesta": "Charles Babbage",
         "dato": "Babbage diseñó la Máquina Analítica en el siglo XIX."},
        {"pregunta": "¿Qué matemática escribió notas que hoy se consideran el primer algoritmo?",
         "opciones": ["Grace Hopper", "Ada Lovelace", "Katherine Johnson", "Barbara Liskov"],
         "respuesta": "Ada Lovelace",
         "dato": "Describió instrucciones para la Máquina Analítica de Babbage."},
        {"pregunta": "¿Cuál fue una de las primeras computadoras electrónicas de propósito general?",
         "opciones": ["ENIAC", "Colossus", "Z3", "Apple I"],
         "respuesta": "ENIAC",
         "dato": "Presentada en 1946; ocupaba una sala completa."},
        {"pregunta": "¿Quién propuso el 'Test de Turing'?",
         "opciones": ["John von Neumann", "Claude Shannon", "Alan Turing", "Norbert Wiener"],
         "respuesta": "Alan Turing",
         "dato": "Evalúa si una máquina puede exhibir inteligencia humana."},
        {"pregunta": "¿Qué empresa lanzó MS-DOS?",
         "opciones": ["IBM", "Microsoft", "Apple", "DEC"],
         "respuesta": "Microsoft",
         "dato": "Clave en la popularización de las PC en los 80."},
    ]

    total = len(preguntas)
    orden = list(range(total))
    if SHUFFLE_QUESTIONS:
        random.shuffle(orden)

    pos = 0
    puntaje = 0
    respondido = False
    opcion_correcta = ""
    botones_opciones: list[ft.ElevatedButton] = []

    # --- Estilos ---
    def chip(text: str, bgcolor=PRIMARY, color=TEXT_PRIMARY):
        return ft.Container(
            content=ft.Text(text, size=12, weight=ft.FontWeight.W_600, color=color),
            bgcolor=bgcolor,
            padding=ft.padding.symmetric(6, 10),
            border_radius=20
        )

    base_btn_style = ft.ButtonStyle(
        padding=ft.padding.symmetric(14, 18),
        shape=ft.RoundedRectangleBorder(radius=14),
        elevation=4,
        bgcolor=ft.Colors.WHITE,
        color=TEXT_DARK,
        side=ft.BorderSide(1, ft.Colors.with_opacity(0.12, ft.Colors.BLACK)),
        animation_duration=250,
    )

    next_btn_style = ft.ButtonStyle(
        padding=ft.padding.symmetric(14, 22),
        shape=ft.RoundedRectangleBorder(radius=18),
        elevation=6,
        bgcolor=PRIMARY,
        color=TEXT_PRIMARY,
    )

    # Fondo con gradiente
    bg = ft.Container(
        expand=True,
        gradient=ft.LinearGradient(
            begin=ft.alignment.top_left,
            end=ft.alignment.bottom_right,
            colors=[ft.Colors.INDIGO_800, ft.Colors.PURPLE_700, ft.Colors.PINK_600]
        ),
    )

    titulo = ft.Text("💻 Quiz: Historia de la Informática",
                     size=28, weight=ft.FontWeight.BOLD, color=TEXT_DARK)
    contador = ft.Text("", size=14, color=ft.Colors.GREY_700)
    aciertos_chip = chip("Aciertos 0", bgcolor=ACCENT, color=ft.Colors.BLACK)

    barra = ft.ProgressBar(
        value=0.0,
        width=520,
        bar_height=10,
        color=ACCENT,
        bgcolor=ft.Colors.with_opacity(0.15, ft.Colors.WHITE),
    )

    pregunta_texto = ft.Text("", size=22, weight=ft.FontWeight.W_700, color=TEXT_DARK)
    dato_curioso = ft.Text("", size=14, italic=True, color=PRIMARY_DARK)
    resultado_texto = ft.Text("", size=18)

    opciones_col = ft.Column(spacing=12, animate_opacity=300)
    siguiente_btn = ft.ElevatedButton("➡️ Siguiente", style=next_btn_style, disabled=True)
    reiniciar_btn = ft.ElevatedButton("🔁 Reiniciar", on_click=lambda e: reiniciar())

    header_row = ft.Row(
        [contador, aciertos_chip],
        alignment=ft.MainAxisAlignment.SPACE_BETWEEN
    )

    card = ft.Container(
        content=ft.Column(
            controls=[
                titulo, header_row, barra, ft.Divider(height=8),
                pregunta_texto, opciones_col,
                resultado_texto, dato_curioso,
                ft.Row([siguiente_btn, reiniciar_btn],
                       alignment=ft.MainAxisAlignment.CENTER, spacing=16),
            ],
            tight=True,
            spacing=16,
        ),
        bgcolor=ft.Colors.WHITE,
        padding=24,
        border_radius=24,
        width=min(page.width, 640),
        shadow=ft.BoxShadow(blur_radius=26, color=ft.Colors.with_opacity(0.25, ft.Colors.BLACK)),
    )

    shell = ft.Stack([bg, ft.Container(content=ft.Column([card],
                    alignment=ft.MainAxisAlignment.CENTER), expand=True, padding=20)])
    page.add(shell)

    # --- Funciones ---
    def cargar_pregunta():
        nonlocal respondido, opcion_correcta, botones_opciones
        respondido = False
        siguiente_btn.disabled = True
        resultado_texto.value = ""
        dato_curioso.value = ""

        idx_real = orden[pos]
        registro = preguntas[idx_real]
        pregunta_texto.value = registro["pregunta"]
        contador.value = f"Pregunta {pos + 1} de {total}"
        aciertos_chip.content = ft.Text(f"Aciertos {puntaje}", size=12)
        barra.value = pos / total

        opciones = list(registro["opciones"])
        if SHUFFLE_OPTIONS:
            random.shuffle(opciones)

        opcion_correcta = registro["respuesta"]
        botones_opciones = []

        for opt in opciones:
            btn = ft.ElevatedButton(text=opt, style=base_btn_style)
            btn.on_click = lambda e, b=btn: verificar_respuesta(b)
            botones_opciones.append(btn)

        opciones_col.controls = botones_opciones
        page.update()

    def colorear(btn, bg, border):
        btn.style = ft.ButtonStyle(
            bgcolor=bg,
            color=ft.Colors.BLACK if bg != BAD else ft.Colors.WHITE,
            side=ft.BorderSide(2, border),
        )

    def verificar_respuesta(btn):
        nonlocal respondido, puntaje
        if respondido:
            return
        seleccion = btn.text
        respondido = True
        for b in botones_opciones:
            b.disabled = True

        if seleccion == opcion_correcta:
            colorear(btn, OK, ft.Colors.GREEN_700)
            resultado_texto.value = "✅ ¡Correcto!"
            puntaje += 1
        else:
            colorear(btn, BAD, ft.Colors.RED_700)
            resultado_texto.value = f"❌ Incorrecto. Era: {opcion_correcta}"
            for b in botones_opciones:
                if b.text == opcion_correcta:
                    colorear(b, OK, ft.Colors.GREEN_700)

        dato_curioso.value = f"ℹ️ {preguntas[orden[pos]]['dato']}"
        aciertos_chip.content = ft.Text(f"Aciertos {puntaje}", size=12)
        siguiente_btn.disabled = False
        page.update()

    def siguiente(_=None):
        nonlocal pos
        if not respondido:
            return
        if pos + 1 < total:
            pos += 1
            cargar_pregunta()
        else:
            fin()

    def fin():
        page.clean()
        page.add(
            ft.Container(
                content=ft.Column(
                    [
                        ft.Text("🏁 Fin del Quiz", size=30, weight=ft.FontWeight.BOLD),
                        ft.Text(f"Puntaje final: {puntaje}/{total}", size=22),
                        ft.ProgressBar(value=1.0, width=400, color=ACCENT),
                        ft.ElevatedButton("🔁 Volver a jugar", style=next_btn_style, on_click=lambda e: reiniciar())
                    ],
                    alignment=ft.MainAxisAlignment.CENTER,
                    horizontal_alignment=ft.CrossAxisAlignment.CENTER,
                ),
                bgcolor=ft.Colors.WHITE,
                padding=30,
                border_radius=20,
                shadow=ft.BoxShadow(blur_radius=25, color=ft.Colors.with_opacity(0.3, ft.Colors.BLACK))
            )
        )
        page.update()

    def reiniciar():
        nonlocal pos, puntaje, orden
        pos = 0
        puntaje = 0
        random.shuffle(orden)
        page.clean()
        page.add(shell)
        cargar_pregunta()

    siguiente_btn.on_click = siguiente
    cargar_pregunta()


if __name__ == "__main__":
    ft.app(target=main)
